<style>
  h1, h2, h3, h4, h5, h6 {
    margin-top: 10px;
    margin-bottom: 10px;
  }
</style>

## **Project:** Requests, JSON, and NLP

### **Student Name:** Brett Neely
#### **GitHub Link:** https://github.com/bncodes19/json-sentiment
#### **Course:** Web Mining and Applied NLP (44-620)

#### **Overview:** Four tasks are assigned to work with the lyrics.ovh public api and write lyric data to json files then perform sentiment analysis on the lyrics.

##### 1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [65]:
import requests
import json

result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

lyrics = "lyrics.json"

with open(lyrics, 'w') as json_file:
    json.dump(result, json_file, indent=4)

##### 2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotation?  Answer this question in a comment in your code cell.

In [66]:
# Print the lyrics of the song from the json file
with open('lyrics.json', 'r') as read_json:
    data = json.load(read_json)
print(data['lyrics'])

I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not actually your friend 
But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I have a secret to tell 

From my electrical well 

It's a simple message and I'm leaving out the whistles and bells 

So the room must listen to me 

Filibuster vigilantly 

My name is blue canary one note* spelled l-i-t-e 

My story's infinite 

Like the Longines Symphonette it doesn't rest 



Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I'm your only friend 

I'm not your only friend 

But I'm a little glowing friend 

But really I'm not actually your f

In [67]:
# Use spaCyTextBlob to perform sentiment analysis on the lyrics and print the polarity score
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

# A fix to help ensure sentiment is set in the next cell
if not spacy.tokens.Span.has_extension('sentiment'):
    spacy.tokens.Span.set_extension('sentiment', getter=lambda x: x._.blob.sentiment_assessments)

text = data['lyrics']

In [68]:
# Sentiment and Polarity analysis
doc = nlp(text)

# Loop through each sentence and identify sentiment, subjectivity, and polarity
for sentence in doc.sents:
    sentiment_blob = sentence._.blob
    assessments = sentiment_blob.sentiment_assessments

    print(f"Sentence: {sentence.text}")
    print(f"Sentiment: {assessments}")
    print("-" * 50)


Sentence: I'm your only friend 
I'm not your only friend 

Sentiment: Sentiment(polarity=0.0, subjectivity=1.0, assessments=[(['only'], 0.0, 1.0, None), (['only'], 0.0, 1.0, None)])
--------------------------------------------------
Sentence: But I'm a little glowing friend 

Sentiment: Sentiment(polarity=-0.1875, subjectivity=0.5, assessments=[(['little'], -0.1875, 0.5, None)])
--------------------------------------------------
Sentence: But really I'm not actually your friend 

Sentiment: Sentiment(polarity=0.0, subjectivity=0.1, assessments=[(['really', 'not', 'actually'], -0.0, 0.1, None)])
--------------------------------------------------
Sentence: But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 


Sentiment: Sentiment(polarity=0.15729166666666666, subjectivity=0.44999999999999996, assessments=[(['blue'], 0.0, 0.1, None), (['light'], 0.4, 0.7, None), (['little'], -0.1875, 0

##### Question: do the lyrics have a more positive or negative connotation?
Analyzing the sentiment analysis per sentence, the lyrics seems to have a positive connotation indicated by words such as fine, blue, light, near, free, etc. 

In [72]:
# Polarity of the entire lyrics
round(doc._.blob.polarity, 4)

0.0451

##### 3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [70]:
# Create a function to download lyrics from the ovh api and save to a json file
def fetch_and_download_lyrics_to_json(artist, song, jsonfile):
    url = f'https://api.lyrics.ovh/v1/{artist}/{song}'    
    response = requests.get(url)
    if response.status_code == 200:
        lyrics = response.json()
        with open(jsonfile, 'w') as file:
            json.dump(lyrics, file, indent=4)
        print(f"{song} by {artist} successfully saved to {jsonfile}.")
    else:
        print(f"Error: Song '{song}' by {artist} not found (status code: {response.status_code})")

fetch_and_download_lyrics_to_json('The Cranberries', 'Zombie', 'zombie.json')
fetch_and_download_lyrics_to_json('Alien Ant Farm', 'Smooth Criminal', 'smooth_criminal.json')
fetch_and_download_lyrics_to_json('Paul Simon', 'You Can Call Me Al', 'you_can_call_me_al.json')
fetch_and_download_lyrics_to_json('Van Morrison', 'Brown Eyed Girl', 'brown_eyed_girl.json')

Zombie by The Cranberries successfully saved to zombie.json.
Smooth Criminal by Alien Ant Farm successfully saved to smooth_criminal.json.
You Can Call Me Al by Paul Simon successfully saved to you_can_call_me_al.json.
Brown Eyed Girl by Van Morrison successfully saved to brown_eyed_girl.json.


##### 4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [71]:
def perform_sentiment_analysis(jsonfile):
    with open(jsonfile, 'r') as read_lyrics:
        lyrics = json.load(read_lyrics)
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    text = lyrics['lyrics']
    doc = nlp(text)
    print(f"The polarity of {jsonfile} is {round(doc._.blob.polarity, 4)}")

json_song_list = ['brown_eyed_girl.json', 'smooth_criminal.json', 'you_can_call_me_al.json', 'zombie.json']

for i in json_song_list:
    perform_sentiment_analysis(i)

The polarity of brown_eyed_girl.json is -0.0896
The polarity of smooth_criminal.json is 0.3784
The polarity of you_can_call_me_al.json is 0.0197
The polarity of zombie.json is -0.0667


##### Question: does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?

For Brown Eyed Girl, I do not think the polarity matches the connotation as the negative polarity indicates a negative sentiment. I perceive this song with a positive sentiment, however there are undertones of melancholy that may contribute to the negative polarity.

For Smooth Criminal and You Can Call Me Al, they both received a positive polarity. This makes sense as both songs are upbeat with positive words.

For Zombie, I think the negative polarity accurately captures the connotation of the song. The song definitely has a negative sentiment.